The json file contains metadata from the website https://facethefuture.com/#projects ,and the scripts shows preprocessing and shapefile generation

In [21]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
df_ftf_2 = pd.read_json("../input/FaceTheFuture/FaceTheFuture.json")
df_ftf_2.tail()

,type,id,geometry,properties
391,Feature,88,"{'type': 'MultiPolygon', 'coordinates': [[[[33...","{'OBJECTID': 88, 'GlobalID': 'a89bfe12-fa97-4c..."
392,Feature,89,"{'type': 'Polygon', 'coordinates': [[[3372119....","{'OBJECTID': 89, 'GlobalID': '00b179df-0cec-48..."
393,Feature,91,"{'type': 'Polygon', 'coordinates': [[[3375034....","{'OBJECTID': 91, 'GlobalID': 'd4c06d5c-5dad-41..."
394,Feature,95,"{'type': 'MultiPolygon', 'coordinates': [[[[33...","{'OBJECTID': 95, 'GlobalID': 'fc228588-6da3-4d..."
395,Feature,97,"{'type': 'MultiPolygon', 'coordinates': [[[[33...","{'OBJECTID': 97, 'GlobalID': '095bb1a2-3007-44..."


In [22]:
# Expanding the  'properties' column to extract information in it in separate columns
properties_df = df_ftf_2["properties"].apply(pd.Series)


df_expanded = pd.concat([df_ftf_2.drop(columns=["properties"]), properties_df], axis=1)


df_expanded.tail()

,type,id,geometry,FID,OBJECTID,GlobalID,Creator,CreationDate,LastEditor,LastEditDate,...,PlantingSeason,ContractPartner,ContractArea,PlantingDensity,Shape__Area,Shape__Length,TreeNumber,PlantingYear_no,Operator,Comments
391,Feature,88,"{'type': 'MultiPolygon', 'coordinates': [[[[33...",NaN,88.0,a89bfe12-fa97-4c45-80ae-bd30d83621ce,FaceTheFuture,1.639063e+12,FaceTheFuture,1.639063e+12,...,Sept-Oct 2021,TreesForAll,48.0,2500.0,0.000000,0.0746,120000.0,2021.0,Kamara Patrick,NaN
392,Feature,89,"{'type': 'Polygon', 'coordinates': [[[3372119....",NaN,89.0,00b179df-0cec-48e4-bfb5-d139f58c724e,FaceTheFuture,1.644654e+12,FaceTheFuture,1.688721e+12,...,April may,Greenchoice,4.0,400.0,0.000003,0.0138,1600.0,2014.0,Kamara Patrick,NaN
393,Feature,91,"{'type': 'Polygon', 'coordinates': [[[3375034....",NaN,91.0,d4c06d5c-5dad-4101-95c2-98031bd4c9b5,FaceTheFuture,1.652693e+12,FaceTheFuture,1.688721e+12,...,APRI_MAY,Greenchoice,50.0,1111.0,0.000000,0.0388,55555.0,2022.0,Kamara Patrick,NaN
394,Feature,95,"{'type': 'MultiPolygon', 'coordinates': [[[[33...",NaN,95.0,fc228588-6da3-4d2e-af30-28ad694ca27f,FaceTheFuture,1.667296e+12,FaceTheFuture,1.688721e+12,...,October,Greenchoice,50.0,1111.0,0.000000,0.0585,55555.0,2022.0,Kamara Patrick,NaN
395,Feature,97,"{'type': 'MultiPolygon', 'coordinates': [[[[33...",NaN,97.0,095bb1a2-3007-4480-a003-ef36d02bf4d5,FaceTheFuture,1.684141e+12,FaceTheFuture,1.688721e+12,...,April-May,PrimaKlima,50.0,1111.0,0.000000,0.0635,55555.0,2023.0,Kamara Patrick,NaN


In [24]:


# Filtering the specified columns with important information for our analysis
filtered_columns = ["GlobalID", "id", "TreeNumber", "ContractArea", "PlantingYear_no", "Comments", "Creator", "geometry"]
df_filtered = df_expanded[filtered_columns]


df_filtered.tail()

,GlobalID,id,TreeNumber,ContractArea,PlantingYear_no,Comments,Creator,geometry
391,a89bfe12-fa97-4c45-80ae-bd30d83621ce,88,120000.0,48.0,2021.0,NaN,FaceTheFuture,"{'type': 'MultiPolygon', 'coordinates': [[[[33..."
392,00b179df-0cec-48e4-bfb5-d139f58c724e,89,1600.0,4.0,2014.0,NaN,FaceTheFuture,"{'type': 'Polygon', 'coordinates': [[[3372119...."
393,d4c06d5c-5dad-4101-95c2-98031bd4c9b5,91,55555.0,50.0,2022.0,NaN,FaceTheFuture,"{'type': 'Polygon', 'coordinates': [[[3375034...."
394,fc228588-6da3-4d2e-af30-28ad694ca27f,95,55555.0,50.0,2022.0,NaN,FaceTheFuture,"{'type': 'MultiPolygon', 'coordinates': [[[[33..."
395,095bb1a2-3007-4480-a003-ef36d02bf4d5,97,55555.0,50.0,2023.0,NaN,FaceTheFuture,"{'type': 'MultiPolygon', 'coordinates': [[[[33..."


In [25]:
# Renaming the columns
new_column_names = {
    "GlobalID": "project_id_reported",
    "id": "site_id_reported",
    "TreeNumber": "trees_planted_reported",
    "ContractArea": "site_sqkm",
    "PlantingYear_no": "planting_date_reported",
    "Comments": "description_reported",
    "Creator": "host_name",
    "geometry": "geometry_reported"
}

df_filtered = df_filtered.rename(columns=new_column_names)

df_filtered.tail()

,project_id_reported,site_id_reported,trees_planted_reported,site_sqkm,planting_date_reported,description_reported,host_name,geometry_reported
391,a89bfe12-fa97-4c45-80ae-bd30d83621ce,88,120000.0,48.0,2021.0,NaN,FaceTheFuture,"{'type': 'MultiPolygon', 'coordinates': [[[[33..."
392,00b179df-0cec-48e4-bfb5-d139f58c724e,89,1600.0,4.0,2014.0,NaN,FaceTheFuture,"{'type': 'Polygon', 'coordinates': [[[3372119...."
393,d4c06d5c-5dad-4101-95c2-98031bd4c9b5,91,55555.0,50.0,2022.0,NaN,FaceTheFuture,"{'type': 'Polygon', 'coordinates': [[[3375034...."
394,fc228588-6da3-4d2e-af30-28ad694ca27f,95,55555.0,50.0,2022.0,NaN,FaceTheFuture,"{'type': 'MultiPolygon', 'coordinates': [[[[33..."
395,095bb1a2-3007-4480-a003-ef36d02bf4d5,97,55555.0,50.0,2023.0,NaN,FaceTheFuture,"{'type': 'MultiPolygon', 'coordinates': [[[[33..."


In [26]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 396 entries, 0 to 395
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   project_id_reported     74 non-null     object 
 1   site_id_reported        396 non-null    int64  
 2   trees_planted_reported  74 non-null     float64
 3   site_sqkm               74 non-null     float64
 4   planting_date_reported  74 non-null     float64
 5   description_reported    1 non-null      object 
 6   host_name               74 non-null     object 
 7   geometry_reported       396 non-null    object 
dtypes: float64(3), int64(1), object(4)
memory usage: 27.8+ KB


In [27]:

df_filtered['url'] = 'https://facethefuture.com/'
df_filtered = df_filtered.assign(species_count_reported=None,country=None, species_planted_reported=None,survival_rate_reported=None)


In [28]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 396 entries, 0 to 395
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   project_id_reported       74 non-null     object 
 1   site_id_reported          396 non-null    int64  
 2   trees_planted_reported    74 non-null     float64
 3   site_sqkm                 74 non-null     float64
 4   planting_date_reported    74 non-null     float64
 5   description_reported      1 non-null      object 
 6   host_name                 74 non-null     object 
 7   geometry_reported         396 non-null    object 
 8   url                       396 non-null    object 
 9   species_count_reported    0 non-null      object 
 10  country                   0 non-null      object 
 11  species_planted_reported  0 non-null      object 
 12  survival_rate_reported    0 non-null      object 
dtypes: float64(3), int64(1), object(9)
memory usage: 43.3+ KB


In [29]:
df_filtered.to_csv('../input/FaceTheFuture/face_the_future.csv', index=False)